In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from glob import glob
import scipy
from statsmodels.stats.proportion import proportion_confint

In [2]:
path_zvk1 = r'C:\Users\Alexei\Desktop\conv\data\26_30'
path_zvk2 = r'C:\Users\Alexei\Desktop\conv\data\02_06'

In [3]:
def period(begin, end):
    return pd.to_datetime(begin, dayfirst = True), pd.to_datetime(end, dayfirst = True) + pd.to_timedelta('1 day')

def merge_zvk(zvk):
    columns = ['№ заявки', 'Дата создания заявки', 'Моб. телефон']

    df_zvk = pd.read_csv(zvk[0], sep = ';', encoding = 'cp1251', dtype = {'Моб. телефон': 'str'})
    df_zvk = df_zvk.reindex(columns = columns)

    for i in range(1, len(zvk)):
        tmp = pd.read_csv(zvk[i], sep = ';', encoding = 'cp1251', dtype = {'Моб. телефон': 'str'})
        tmp = tmp.reindex(columns = columns)
        df_zvk = df_zvk.append(tmp, ignore_index = True)

    df_zvk.dropna(inplace = True)
    df_zvk.drop_duplicates(inplace = True)
    df_zvk['Дата создания заявки'] = list(map(lambda x: pd.to_datetime(str(x).split(' ')[0], dayfirst = True),
                                                                                      df_zvk['Дата создания заявки']))
    return df_zvk

def merge_abnd(abnd):
    usecols = [0, 1]
    names = ['Моб. телефон', 'Дата']
    df_abnd = pd.read_csv(abnd[0], sep = ';', names = names, usecols = usecols, encoding = 'cp1251',
                                                                      dtype = {'Моб. телефон': 'str'})
    for i in range(1, len(abnd)):
        tmp = pd.read_csv(abnd[i], sep = ';', names = names, usecols = usecols, encoding = 'cp1251',
                                                                      dtype = {'Моб. телефон': 'str'})
        df_abnd = df_abnd.append(tmp, ignore_index = True)

    df_abnd.dropna(inplace = True)
    df_abnd.drop_duplicates(inplace = True)
    df_abnd['Дата'] = list(map(lambda x: pd.to_datetime(str(x).split(' ')[0], dayfirst = True), df_abnd['Дата']))
    return df_abnd

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1[0]) + len(sample1[1])
    n2 = len(sample2[0]) + len(sample2[1])
    
    p1 = len(sample1[1]) / n1
    p2 = len(sample2[1]) / n2
    
    P = float(p1 * n1 + p2 * n2) / (n1 + n2)
    
    return (p1 - p2) / (P * (1 - P) * (1 / n1 + 1 / n2))**0.5

In [4]:
f = glob(path_zvk1 + '\\' + '*.csv')
zvk1 = [i for i in f if 'five' in i]
abnd1 = list(set(f) - set(zvk1))

g = glob(path_zvk2 + '\\' + '*.csv')
zvk2 = [i for i in g if 'five' in i]
abnd2 = list(set(g) - set(zvk2))

df_zvk1 = merge_zvk(zvk1)
df_zvk2 = merge_zvk(zvk2)
df_abnd1 = merge_abnd(abnd1)
df_abnd2 = merge_abnd(abnd2)

In [5]:
begin1, end1 = period('26.08.2019', '29.08.2019')
begin2, end2 = period('03.09.2019', '06.09.2019')

df_zvk1_ = df_zvk1[(df_zvk1['Дата создания заявки'] >= begin1) & (df_zvk1['Дата создания заявки'] < end1)]
df_zvk2_ = df_zvk2[(df_zvk2['Дата создания заявки'] >= begin2) & (df_zvk2['Дата создания заявки'] < end2)]
df_abnd1_ = df_abnd1[(df_abnd1['Дата'] >= begin1) & (df_abnd1['Дата'] < end1)]
df_abnd2_ = df_abnd2[(df_abnd2['Дата'] >= begin2) & (df_abnd2['Дата'] < end2)]

print('Период 1 составляют отчетные даты: {}\n'.format(sorted(set(df_zvk1_['Дата создания заявки']) & set(df_abnd1_['Дата']))))
print('Период 2 составляют отчетные даты: {}'.format(sorted(set(df_zvk2_['Дата создания заявки']) & set(df_abnd2_['Дата']))))

Период 1 составляют отчетные даты: [Timestamp('2019-08-26 00:00:00'), Timestamp('2019-08-27 00:00:00'), Timestamp('2019-08-28 00:00:00'), Timestamp('2019-08-29 00:00:00')]

Период 2 составляют отчетные даты: [Timestamp('2019-09-03 00:00:00'), Timestamp('2019-09-04 00:00:00'), Timestamp('2019-09-05 00:00:00'), Timestamp('2019-09-06 00:00:00')]


In [6]:
abnd1_s = set(df_abnd1_['Моб. телефон']) - (set(df_abnd1_['Моб. телефон']) & set(df_zvk1_['Моб. телефон']))
zvk1_s  = set(df_zvk1_['Моб. телефон'])
conv1 = len(zvk1_s) / (len(zvk1_s) + len(abnd1_s))
print('Конверсия в период с {} по {}: {}%'.format(begin1, end1, round(conv1 * 100, 2)))

abnd2_s = set(df_abnd2_['Моб. телефон']) - (set(df_abnd2_['Моб. телефон']) & set(df_zvk2_['Моб. телефон']))
zvk2_s  = set(df_zvk2_['Моб. телефон'])
conv2 = len(zvk2_s) / (len(zvk2_s) + len(abnd2_s))
print('Конверсия в период с {} по {}: {}%'.format(begin2, end2, round(conv2 * 100, 2)))

Конверсия в период с 2019-08-26 00:00:00 по 2019-08-30 00:00:00: 69.93%
Конверсия в период с 2019-09-03 00:00:00 по 2019-09-07 00:00:00: 68.14%


In [7]:
z_stat = proportions_diff_z_stat_ind([abnd1_s, zvk1_s], [abnd2_s, zvk2_s])
2 * (1 - scipy.stats.norm.cdf(abs(z_stat)))

0.03534555583782262